In [ ]:
import pydeck as pdk
import pandas as pd
import os
import requests

In [ ]:
url = 'http://openAPI.seoul.go.kr:8088/5356624d637a686f36396b77634850/json/bikeStationMaster/1/3/'
tmp_ = requests.get(url).json()
api_cols = pd.DataFrame(tmp_['bikeStationMaster']['row']).columns

aa = [x*1000 for x in range(1,5)]

BS_df = pd.DataFrame(columns = api_cols)

for i in range(len(aa)):
    start_num = aa[i]-999
    end_num = aa[i]

    url = f'http://openAPI.seoul.go.kr:8088/5356624d637a686f36396b77634850/json/bikeStationMaster/{start_num}/{end_num}/'

    url2json = requests.get(url).json()
    data = pd.DataFrame(url2json['bikeStationMaster']['row'])

    BS_df = pd.concat([BS_df, data], axis=0)

In [ ]:
BS_df.reset_index(drop=True, inplace = True)

In [ ]:
bs_df_master = pd.read_csv('./tpss_bcycl_od_statnhm_20230801.csv', encoding='cp949')

In [ ]:
bs_df_master = bs_df_master[(bs_df_master['시작_대여소_ID']!=bs_df_master['종료_대여소_ID']) & (bs_df_master['전체_이용_분']!=0)& (bs_df_master['전체_이용_거리']!=0)]

In [ ]:
display(BS_df.head())
display(bs_df_master.head())

In [ ]:
tmp_df = pd.DataFrame(columns = ['start_id','start_lng','start_lat','end_id','end_lng','end_lat'])
tmp_df['start_id'] = bs_df_master['시작_대여소_ID']
tmp_df['end_id'] = bs_df_master['종료_대여소_ID']
# tmp_df.drop_duplicates(inplace = True)
tmp_df.reset_index(drop=True, inplace = True)

In [ ]:
merge_left = pd.merge(tmp_df,BS_df, how='left', left_on='start_id', right_on='LENDPLACE_ID')

In [ ]:
merge_left_ = pd.merge(tmp_df,BS_df, how='left', left_on='end_id', right_on='LENDPLACE_ID')

In [ ]:
tmp_df['start_lng'] = merge_left['STATN_LNT']
tmp_df['start_lat'] = merge_left['STATN_LAT']

tmp_df['end_lng'] = merge_left_['STATN_LNT']
tmp_df['end_lat'] = merge_left_['STATN_LAT']

In [ ]:
bs_df_master.reset_index(drop= True,inplace=True)

In [ ]:
tmp_df['cnt'] = bs_df_master['전체_건수']
tmp_df['time'] = bs_df_master['전체_이용_분']


In [ ]:
tmp_df

In [ ]:
tmp_df = tmp_df[tmp_df['end_id']!='X']
tmp_df.reset_index(drop = True, inplace = True)

In [ ]:
tmp_df['norm_cnt'] = tmp_df['cnt'] / tmp_df['cnt'].max()
tmp_df['norm_time'] = tmp_df['time'] / tmp_df['time'].max()

In [ ]:
bs_df_master[bs_df_master['시작_대여소_ID']=='ST-2705']

In [ ]:
tmp_df[tmp_df['end_lng'].isnull() == True]

In [ ]:
tmp_df = tmp_df[tmp_df['end_id']!='X']
tmp_df.reset_index(drop = True, inplace = True)

In [ ]:
layer = pdk.Layer(
    'LineLayer',
    tmp_df,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    get_width='1 + 10 * norm_cnt',
    get_color='[255, 255 * norm_time, 120]',
    pickable=True,
    auto_highlight=True
)

# pydeck.data_utils.compute_view 는 Points 들의 경도, 위도를 리스트로 주면, 알아서 view_state 를 만들어줍니다.
view_state = pdk.data_utils.compute_view(tmp_df[['start_lng', 'start_lat']].values)
view_state.zoom = 13

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
tmp_df['start_id'].value_counts()

In [ ]:
tmp_df_01 = tmp_df[tmp_df['start_id'] == 'ST-2031']

In [ ]:
tmp_df_01['norm_cnt'] = tmp_df_01['cnt'] / tmp_df_01['cnt'].max()
tmp_df_01['norm_time'] = tmp_df_01['time'] / tmp_df_01['time'].max()

In [ ]:
tmp_df_02 = tmp_df_01.groupby(['start_id','end_id','start_lng','start_lat','end_lng','end_lat']).sum().reset_index().iloc[:, :-2]

In [ ]:
tmp_df_02['norm_cnt'] = tmp_df_02['cnt'] / tmp_df_02['cnt'].max()
tmp_df_02['norm_time'] = tmp_df_02['time'] / tmp_df_02['time'].max()

In [ ]:
layer = pdk.Layer(
    'LineLayer',
    tmp_df_02,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    get_width='1 + 10 * norm_cnt',
    get_color='[255, 255 * norm_time, 120]',
    pickable=True,
    auto_highlight=True
)

# pydeck.data_utils.compute_view 는 Points 들의 경도, 위도를 리스트로 주면, 알아서 view_state 를 만들어줍니다.
view_state = pdk.data_utils.compute_view(tmp_df_02[['start_lng', 'start_lat']].values)
view_state.zoom = 13

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'ArcLayer',
    tmp_df_02,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    get_width='1 + 10 * norm_cnt',
    get_source_color='[255,  255 * norm_cnt, 120]',
    get_target_color='[255,  255 * norm_cnt, 120]',
    pickable=True,
    auto_highlight=True
)

# pydeck.data_utils.compute_view 는 Points 들의 경도, 위도를 리스트로 주면, 알아서 view_state 를 만들어줍니다.
view_state = pdk.data_utils.compute_view(tmp_df_02[['start_lng', 'start_lat']].values)
view_state.zoom = 12
view_state.bearing = -15
view_state.pitch = 45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
tmp_df_02['cnt'].max()

In [ ]:
tmp_df_02[tmp_df_02['cnt']==86]

In [ ]:
BS_df[(BS_df['LENDPLACE_ID']=='ST-2033') | (BS_df['LENDPLACE_ID']=='ST-2031')]

In [ ]:
tmp_df_03 = tmp_df[tmp_df['end_id'] == 'ST-2031']

In [ ]:
tmp_df_03['norm_cnt'] = tmp_df_03['cnt'] / tmp_df_03['cnt'].max()
tmp_df_03['norm_time'] = tmp_df_03['time'] / tmp_df_03['time'].max()

In [ ]:
tmp_df_04 = tmp_df_03.groupby(['start_id','end_id','start_lng','start_lat','end_lng','end_lat']).sum().reset_index().iloc[:, :-2]

In [ ]:
tmp_df_04['norm_cnt'] = tmp_df_04['cnt'] / tmp_df_04['cnt'].max()
tmp_df_04['norm_time'] = tmp_df_04['time'] / tmp_df_04['time'].max()

In [ ]:
layer = pdk.Layer(
    'ArcLayer',
    tmp_df_02,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    get_width='1 + 10 * norm_cnt',
    get_source_color='[255,  255 * norm_cnt, 120]',
    get_target_color='[255,  255 * norm_cnt, 120]',
    pickable=True,
    auto_highlight=True
)
layer2 = pdk.Layer(
    'ArcLayer',
    tmp_df_04,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    get_width='1 + 10 * norm_cnt',

    get_source_color = '[120,255,255]',
    get_target_color = '[120,255,255]',

    # get_source_color='[120,  120, 255 * norm_cnt]',
    # get_target_color='[120,  120, 255 * norm_cnt]',
    pickable=True,
    auto_highlight=True
)

# pydeck.data_utils.compute_view 는 Points 들의 경도, 위도를 리스트로 주면, 알아서 view_state 를 만들어줍니다.
view_state = pdk.data_utils.compute_view(tmp_df_02[['start_lng', 'start_lat']].values)
view_state.zoom = 12
view_state.bearing = -15
view_state.pitch = 45

r = pdk.Deck(layers=[layer, layer2], initial_view_state=view_state)
r.to_html()

In [ ]:
tmp_df_05 = tmp_df_04
tmp_df_05['cnt'] = tmp_df_04['cnt'].apply(lambda x : x * -1)

In [ ]:
tmp_df_05.rename(columns={'start_id':'end_id','end_id':'start_id','start_lng':'end_lng','start_lat':'end_lat','end_lng':'start_lng','end_lat':'start_lat'}, inplace=True)

In [ ]:
tmp_df_05.head()

In [ ]:
tmp_df_06 = pd.concat([tmp_df_02,tmp_df_05])

In [ ]:
tmp_df_06

In [ ]:
tmp_df_07 = tmp_df_06.pivot_table(index=['start_id','end_id','start_lng','start_lat','end_lng','end_lat'], aggfunc='sum')

In [ ]:
imp_df_08 = tmp_df_07[tmp_df_07['cnt']<0]
imp_df_09 = tmp_df_07[tmp_df_07['cnt']>=0]

In [ ]:
imp_df_08.reset_index(inplace=True)
imp_df_09.reset_index(inplace=True)

In [ ]:
layer = pdk.Layer(
    'ArcLayer',
    imp_df_08,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    # get_width='1 + 10 * norm_cnt',
    get_source_color='[255, 255, 120]',
    get_target_color='[255, 255, 120]',
    pickable=True,
    auto_highlight=True
)

layer2 = pdk.Layer(
    'ArcLayer',
    imp_df_09,
    get_source_position='[start_lng, start_lat]',
    get_target_position='[end_lng, end_lat]',
    # get_width='1 + 10 * norm_cnt',

    get_source_color = '[120,255,255]',
    get_target_color = '[120,255,255]',

    # get_source_color='[120,  120, 255 * norm_cnt]',
    # get_target_color='[120,  120, 255 * norm_cnt]',
    pickable=True,
    auto_highlight=True
)

# pydeck.data_utils.compute_view 는 Points 들의 경도, 위도를 리스트로 주면, 알아서 view_state 를 만들어줍니다.
view_state = pdk.data_utils.compute_view(imp_df_08[['start_lng', 'start_lat']].values)
view_state.zoom = 12
view_state.bearing = -15
view_state.pitch = 45

r = pdk.Deck(layers=[layer, layer2], initial_view_state=view_state)
r.to_html()